This Jupyter Notebook is meant to be accessed via Google Colab and will be the main method to train models on Google Colab.

In [4]:
# primary importing code block to utilize the python files
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, ZeroPadding2D, BatchNormalization, MaxPool2D
from pydub import AudioSegment   # main class from pydub package used to upload mp3 into Python and then get a NumPy array
import IPython.display as ipd    # ability to play audio in Jupyter Notebooks if needed
import librosa as lb             # loads the librosa package
import librosa.display
from src.configs import *
from src.utils import MusicAlignedTab, create_FullSet_df, one_hot_encode, collapse_class, clean_labels
from src.dataset import Dataset

In [ ]:
mat = MusicAlignedTab('mookies_last_christmas')
df = mat.MAT

In [ ]:
df = clean_labels(df)
MusicAlignedTab.labels_summary(df)
df = collapse_class(df, keep_dynamics = False, keep_bells = False, keep_toms_separate = False, hihat_classes=1, cymbal_classes=1)
MusicAlignedTab.labels_summary(df)

In [ ]:
encode_df = one_hot_encode(df)
print(encode_df.columns)
encode_df.head()

In [ ]:
%time targets, target_dict = create_targets(S, encode_df)

In [ ]:
for idx, val in target_dict.items():
    print(f'total windows that {val} are labeled 1 = {np.count_nonzero(targets[idx,:,0])}')
print(f'total windows = {targets.shape[1]}')

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot()
plt.imshow(targets[:,:,1])
plt.show()

In [ ]:
MusicAlignedTab.labels_summary(encode_df)

In [ ]:
targets = np.zeros((7, 10, 3), dtype=int)
print(targets.shape)
targets[:, 0, :]

In [ ]:
df.drop(columns = ['song slice', 'sample start'])[df.drop(columns = ['song slice', 'sample start']) != '-'].describe()

In [ ]:
mat.random_alignment_checker(['BD', 'SD', 'CC'], 14)

In [ ]:
librosa.display.specshow(S[:,:,0], sr=44100, x_axis='time', y_axis = 'mel')

### FullSet Testing

In [ ]:
def play(samples, sr=44100):
    '''
    Helper function just so I can type play(samples) to get the song output
    '''
    return ipd.Audio(samples, rate = sr)

In [ ]:
FullSet = create_FullSet_df(SONGS_PATH)
FullSet_clean = clean_labels(FullSet)
MusicAlignedTab.labels_summary(FullSet_clean)
FullSet_collapse = collapse_class(FullSet_clean, keep_dynamics = False, keep_bells = False, keep_toms_separate = False, hihat_classes=1, cymbal_classes=1)
MusicAlignedTab.labels_summary(FullSet_collapse)
FullSet_encoded = one_hot_encode(FullSet_collapse)

In [ ]:
tset = Dataset('train', FullSet_encoded)
vset = Dataset('val', FullSet_encoded)

In [ ]:
song_idx = 1

In [ ]:
%%time
song_df = tset.subset_df.loc[tset.song_list[song_idx]].copy()
song = np.vstack(song_df['song slice'].to_numpy()).T   # stacks the song slices back into a single numpy array of shape (channels, samples)
mono_song = lb.core.to_mono(song)
channels = [mono_song]              # channels is a list of either [mono_song] or [mono, L_song, R_song]
if INCLUDE_LR_CHANNELS:             # appending the LR channels to the channels variable
    channels.append(song[0,:])
    channels.append(song[1,:])
aug_channels = tset.augment_audio_cp(channels, tset.aug_comp, sr=SAMPLE_RATE)

In [ ]:
play(aug_channels[0])

In [ ]:
INCLUDE_LR_CHANNELS = False
%time spectrogram, target, target_dict = tset.preprocess_song(tset.song_list[7])

In [ ]:
print(spectrogram.shape)
print(target.shape)
print(target_dict)
tset.song_list[7]

In [ ]:
vset.subset_df

In [ ]:
labels = np.zeros((5,5))

In [ ]:
labels_df = vset.subset_df.loc['four_years'].drop(columns = ['sample start', 'song slice']).copy()

In [ ]:
labels_df

In [ ]:
np.stack([labels_df.loc[0].to_numpy() for _ in range(3)], axis = -1).shape

In [ ]:
targets = np.zeros((7, 15,3))

In [ ]:
%%time
save_spec = []
save_tar = []
for spec, tar in vset:
    save_spec.append(spec)
    save_tar.append(tar)
    print(f'shape of target = {tar.shape}')

In [ ]:
fy_spec = save_spec[0]
mb_spec = save_spec[1]
fy_tar = save_tar[0]
mb_tar = save_tar[1]

In [6]:
numpysave_fp = 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Experimental/'

In [ ]:
np.save(numpysave_fp + 'fy_spec', fy_spec)
np.save(numpysave_fp + 'fy_tar', fy_tar)
np.save(numpysave_fp + 'mb_spec', mb_spec)
np.save(numpysave_fp + 'mb_tar', mb_tar)

In [ ]:
np.savez(numpysave_fp + 'zip_uncomp', fy_spec, fy_tar, mb_spec, mb_tar)

In [ ]:
np.savez_compressed(numpysave_fp + 'zip_comp', fy_spec, fy_tar, mb_spec, mb_tar)

In [9]:
fy_spec = np.load(numpysave_fp+'fy_spec.npy')
fy_tar = np.load(numpysave_fp+'fy_tar.npy')

In [50]:
one_chan = fy_spec[:,:,0]
one_chan.shape
one_tar = fy_tar[:,:,0]

In [62]:
for row in range(one_tar.shape[0]) :
    print(np.count_nonzero(one_tar[row,:]))

990
250
774
609
0
61
1105


In [66]:
df = pd.DataFrame(one_tar.T, columns=['a','b','c','d','e','f','g'])
print(df.head())

   a  b  c  d  e  f  g
0  0  0  1  0  0  0  0
1  0  0  0  0  0  0  0
2  0  0  0  0  0  0  0
3  0  0  0  0  0  0  0
4  0  0  0  0  0  0  0


In [12]:
def expand_spectrogram(spectrogram, pre_context, post_context):
    '''
    Expands a 2D spectrogram into slices of the correct shape to be input into the model
    
    Args:
        spectrogram [np.array]:
        pre_context [int]:
        post_context [int]:
    
    Returns:
        np.array: 
    
    '''
    
    n_features, n_windows = spectrogram.shape
    
    input_width = 1 + pre_context + post_context
    min_value = np.min(spectrogram)
    
    expanded_spectrogram = np.full(shape = (n_windows, n_features, input_width), fill_value = min_value)# assign into this np.array
    
    for idx in range(n_windows):
        if idx - pre_context < 0:    # in a window where you would slice before the beginning
            start = pre_context-idx
            expanded_spectrogram[idx, :,start:] = spectrogram[:, 0:idx+post_context+1 ]
        elif idx + post_context+1 > n_windows: # in a window where you would slice past the end
            end = post_context+1 - (n_windows - idx)
            expanded_spectrogram[idx, :, :input_width-end] = spectrogram[:, idx-pre_context: n_windows ]
        else:    # in a "normal" middle window where you slice into the spectrogram normally
            expanded_spectrogram[idx, :,:] = spectrogram[:, idx-pre_context : idx+post_context+1]
    
    return expanded_spectrogram

In [43]:
exam = np.arange(0,15)
exam = np.stack([exam, exam*2, exam*3], axis=-1).T
print(exam)
print(exam.shape)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  2  4  6  8 10 12 14 16 18 20 22 24 26 28]
 [ 0  3  6  9 12 15 18 21 24 27 30 33 36 39 42]]
(3, 15)


In [48]:
stacked = lb.feature.stack_memory(exam, n_steps=2, delay=2, mode='wrap')
print(stacked.shape)
print(stacked)

(6, 15)
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  2  4  6  8 10 12 14 16 18 20 22 24 26 28]
 [ 0  3  6  9 12 15 18 21 24 27 30 33 36 39 42]
 [13 14  0  1  2  3  4  5  6  7  8  9 10 11 12]
 [26 28  0  2  4  6  8 10 12 14 16 18 20 22 24]
 [39 42  0  3  6  9 12 15 18 21 24 27 30 33 36]]


In [ ]:
%%time
exp_spec = expand_spectrogram(one_chan, 5,5)
exp_spec[10000,0,:]

## Testing of the train.py

In [71]:
from train import *
main()

subdirs = ['C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\ancient_tombs', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\best_of_me', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\boulevard_of_broken_dreams', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\cant_be_saved', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\face_down', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\family_tradition', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\fireworks_at_dawn', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\forever_at_last', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\four_years', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\garden_state', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\gunpowder', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\hair_of_the_dog', 'C:/User

C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


KeyboardInterrupt: 

int1 = 0
int2 = 100
Song = 1
lr= 0.00023453
song_l = 234.4566

In [ ]:
print('Epoch: {:2}/{} Song{:3}, lr:{:.6f}, song_loss:{:8.2f}'.format(int1, int2, Song, lr, song_l))

In [67]:
sp = 'title_song.mp3'

In [69]:
os.path.splitext(sp)[0]

'title_song'